In [242]:
import cobra
import numpy as np
from cobrame.util import building, mu, me_model_interface

## Metabolites

met_output_file = 'metabolites.txt'
m_model_file = './iYO844.json'

m_model = cobra.io.load_json_model(m_model_file)

In [243]:
def duplicates(lst, item, counter):
    return [i for i, x in enumerate(lst) if x == item and i != counter]
def duplicate_indices(lst, item):
    return [i for i, x in enumerate(lst) if x == item]

In [244]:
def convert_compartments(compartment_list):
    compartment_ids = ['c','e','p']
    compartment_equivalence = ['Cytosol','Extra-organism','Periplasm']
    
    for c in range(0,len(compartment_list)):
        if compartment_list[c] in compartment_ids:
            ind = compartment_ids.index(compartment_list[c])
            compartment_list[c] = compartment_equivalence[ind]
            
    return compartment_list

def generate_metfile(m_model,filename):
    file = open(filename,'a')
    
    metabolite_list = [None]*len(m_model.metabolites)
    np.transpose(metabolite_list)
    compartment_list = [None]*len(m_model.metabolites)
    homogeneous_metabolite_list = []
    
    
    for m in range(0,len(m_model.metabolites)):
        ID = m_model.metabolites[m].id
        compartment_list[m] = ID[len(ID)-1]
        metabolite_list[m] = ID[:len(ID)-2]
        
    compartment_list = convert_compartments(compartment_list)
    
    other_mets_index = [];
    for c in range(0,len(m_model.metabolites)):
        if len(duplicates(other_mets_index,c,-1)) == 0:
            met = metabolite_list[c]
            name = m_model.metabolites[c].name
            formula = m_model.metabolites[c].formula

            other_mets_index = duplicates(metabolite_list,met,c)
            
            if formula is None:
                formula = 'no-formula'
            
            string = met + '	' + name + '	' + formula + '	' + compartment_list[c]
            for other_mets in other_mets_index:
                string = string + ' AND ' + compartment_list[other_mets]
            
            string = string + '\n'
            file.write(string)
    file.close()

In [245]:
generate_metfile(m_model,met_output_file)